In [24]:
#Project name: NetCDF --> Excel
#Description: It receives a NetCDF file with a predefined format (should we elaborate here?) and returns the equivalent file in excel format. 
#Programmers: Amir \& Ashley
#Date: 07-09-2020

In [25]:
#Task list
## 1. Make the code more generic by not using the column names
## 2. Annotating the columns in order to keep the column order
## 3. Activate the 2-factor authentication


In [43]:
#Loading the packages
#The openpyxl should also be installed (it is essential for saving the dataframe as an excel file) 
import pandas as pd
import xarray as xr
import numpy as np

In [108]:
#Loading the NetCDF file
# ds_disk = xr.open_dataset("8-Plagioclase-trace-elements.nc")
#sheet names:
#3-WR-Major-and-trace-elements
#4-Amphibole-Major-elements
#5-OPX-Major-elements
#6-Glass-Major-elements
#7-Plagioclase-Major-elements
#8-Plagioclase-trace-elements

sheet_name = "3-WR-Major-and-trace-elements"
ds_disk = xr.open_dataset(sheet_name+".nc")

In [109]:
#Fetching the global attributes
ds_global_attributes = ds_disk.attrs

In [110]:
#Fetching the indexes (It will not be used later on as the indeces are also available in the variables)
ds_indeces = list(ds_disk.coords.indexes["SAMPLE NAME"])

In [111]:
#Fetching the variables
ds_disk_variables = ds_disk.to_dataframe().reset_index()
#Fetching the column names
column_names = list(ds_disk_variables.columns)

In [112]:
# Retrieve the columns order
orders = ds_disk_variables.iloc[len(ds_disk_variables)-1].to_dict()

# Remove the columns order
ds_disk_variables = ds_disk_variables.drop(axis=0, index=len(ds_disk_variables)-1)
# ds_disk_variables = ds_disk_variables.drop(axis=0, index=len(ds_disk_variables))

# Extraction of the order and place them in two arrays
column_dict = {"left":{}, "right":{}}
for i,j in orders.items():
    info = str(j).split(".")
    if j == "flag":
        side, order = 1 , 0
        column_dict["left"][i] = "1.0"
    else:
        side, order = info[0] , info[1]
        if side == "1":
            column_dict["left"][i] = j
        elif side == "2":
            column_dict["right"][i] = j

left_dict = {k: v for k, v in sorted(column_dict["left"].items(), key=lambda item: item[1])}
right_dict = {k: v for k, v in sorted(column_dict["right"].items(), key=lambda item: item[1])}

left = [i for i in left_dict]
right = [i for i in right_dict]

# Reordering the columns
ds_disk_variables = ds_disk_variables[left+right]

In [113]:
# #Reorganising the order of the columns
# left = list(ds_disk_variables.columns)[:10]
# right = list(ds_disk_variables.columns)[10:]
# left = ["SAMPLE NAME", "IGSN", "SPECIES", "sample preparation", "chemical treatment", "description", "grain spot ID", "spot location", "calculated average", "number of replicates"]
# ds_disk_variables = ds_disk_variables[left+right]

In [114]:
#Creating an empty dictionary where the keys are the column names, and the values refer to the properties of those columns 

ds_dict = {i:[] for i in left+right}
for j in left:
    ds_dict[j] = {}
    ds_dict[j]["comment"] = ""

for j in right:
    ds_dict[j] = {}
    ds_dict[j]["units"] = ""
    ds_dict[j]["comment"] = ""

In [115]:
#Populating the dictionary 
for i in left:
    if i == "SAMPLE NAME":
        #Is this property already stored in the NetCDF file?
        ds_dict[i]["comment"] = "must match a sample on the SAMPLES tab column A"
    else:
        ds_dict[i]["comment"] = ds_disk.data_vars[i].attrs["comment"]

        
for j in right:
    ds_dict[j]["units"] = ds_disk.data_vars[j].attrs["units"]
    ds_dict[j]["comment"] = ds_disk.data_vars[j].attrs["comment"]

In [116]:
#Creating a dataframe for the left side of the tables's header
s = pd.DataFrame(columns=ds_disk_variables.columns)
s.loc[0] = [ds_dict[i]["comment"] if i in left else np.nan for i in s.columns] 

g = pd.DataFrame(columns=ds_disk_variables.columns)
g.loc[0] = [i if i in left else np.nan for i in ds_disk_variables.columns]

a1 = pd.concat([g,s])

In [117]:
#Creating a dataframe for the right side of the tables's header

h = pd.DataFrame(columns=ds_disk_variables.columns)
h.loc[0] = [ds_dict[i]["comment"].replace("METHOD CODE: ","") if i in right else np.nan for i in s.columns]
h.loc[1] = [ds_dict[i]["units"] if i in right else np.nan for i in s.columns]

v = pd.DataFrame(columns=ds_disk_variables.columns)
v.loc[0] = [i if i in right else np.nan for i in ds_disk_variables.columns]

a2 = pd.concat([v,h])

In [118]:
#Manual insertion of some values in the table (Do they exist in the NetCDF file? if so, where?)
a2.iloc[0]["number of replicates"] = "PARAMETER [list]"
a2.iloc[1]["number of replicates"] = "METHOD CODE [more info]:"
a2.iloc[2]["number of replicates"] = "UNIT [list]:"

In [119]:
#Here, the header to the table is created
a3 = pd.concat([a2,a1])

In [120]:
#Here, the header and variables are added together
a4 = pd.concat([a3,ds_disk_variables])

In [121]:
#Changing the columns names
k = [np.nan for i in range(len(left+right))]
k[0] = ds_global_attributes["Description"]

a4.columns = k

In [122]:
#Replacing the dataframe index by the column "SAMPLE NAME"
a4.index = a4[ds_global_attributes["Description"]]

#Removing the column "SAMPLE NAME" from the dataframe
a4 = a4.drop(columns=[ds_global_attributes["Description"]])

In [123]:
#Saving the dataframe as an excel file
a4.to_excel(sheet_name+".xlsx", sheet_name=ds_global_attributes["Title"])